#Data Engineering part

In [7]:
import pandas as pd
from datetime import date
from easy_exchange_rates import API

df = pd.read_csv('https://raw.githubusercontent.com/JaniceLibbyThomas/Sample_Dataset/main/Zomato.csv')
# df.to_csv('zomato.csv')
# display(df.head(10))

df1 = pd.read_csv('/content/Country-Code.csv')
display(df1)


api = API()
currency_conversion = list(df1['Currency_code'])
# print(currency_conversion)
indian_exchange_rate =[]

for i in range (0,len(currency_conversion)):
  time_series = api.get_exchange_rates(
  base_currency=currency_conversion[i], 
  start_date=str(date.today()), 
  end_date=str(date.today()), 
  targets=["INR"]
)
  data_frame = api.to_dataframe(time_series)
  indian_exchange_rate.append((time_series[str(date.today())]['INR']))
df1['indian_exchange_rate']=indian_exchange_rate

#Mergining the Calculated exchange rate to DF

df3 = pd.merge(df,df1)

# saving the money --> in INR
ex_rate = list(df3['indian_exchange_rate'])
rate_conversion = list(df3['Average_Cost_for_two'])

Price_in_INR = []
for i in range (0,len(rate_conversion)):
  Price_in_INR.append(ex_rate[i]*rate_conversion[i])

df3['Indian_Rupee']=Price_in_INR

#calculation of average money spent by each country,dine in, delivery,costlier cusine, most popular cusine...

s = list(dict(df3.groupby("Country")['Indian_Rupee'].mean()))
s1 = list(df3.groupby("Country")['Indian_Rupee'].mean())

df4 = pd.DataFrame(s,columns=['Country'])
df4['Average_Money_Spent']=s1

Country_list = list(df4['Country'])
length = len(Country_list)


dine = []
delivery = []
Costlier_Cusine = []
Popular_Cusine = []
max_spent_city = []
min_spent_city = []
min_spent_amount = []
a1 = ''

for i in range(0,length):

#countrywise dataFrame
  specific_country=(df3[ df3['Country']==Country_list[i]])

# to get the dine-in and delivery count of each country
  dine.append(len(list(specific_country[specific_country['Has_Table_booking']=='Yes'].index.values)))
  delivery.append(len(list(specific_country[specific_country['Has_Online_delivery']=='Yes'].index.values)))

#most Costlier cusine

  max_spent = specific_country.Indian_Rupee.max()

  index = list(specific_country[specific_country['Indian_Rupee']==max_spent].index.values)

  Costlier_Cusine.append(specific_country.loc[int(index[0]),'Cuisines'])

#Most popular Cusine

  popular_vote = specific_country.Aggregate_rating.max()

  li = {}

  index = (specific_country[specific_country['Aggregate_rating']==popular_vote].index.values)

  for i in range(0,len(index)):
  # li[int(specific_country.loc[int(index[i]),['Votes']])]=(dict(specific_country.loc[int(index[i]),['Cuisines']]))
    b = (list(specific_country.loc[int(index[i]),['Votes']]))
    a = str(list(specific_country.loc[int(index[i]),['Cuisines']])).replace('[','').replace(']','').replace("'",'')
    li[a] = b[0]

  value = dict(li)

  Popular_Cusine.append(max(zip(value.values(), value.keys()))[1])

#Highest Spending City 

  city = specific_country.City.unique()
  city_wise_cost = (dict(specific_country.groupby("City")['Indian_Rupee'].sum()))
  max_spent_city.append(max(zip(city_wise_cost.values(), city_wise_cost.keys()))[1])

#Min Spending City

  min_spent_city.append(min(zip(city_wise_cost.values(), city_wise_cost.keys()))[1])
  # min_spent_amount.append(city_wise_cost[(min_spent_city[0])])

# Appending the data to Df4

df4['Dine-In'] = dine
df4['Online_Delivery'] = delivery
df4['Popular_Cusine'] = Popular_Cusine
df4['Costlier_Cusine'] = Costlier_Cusine
df4['Max_spent_city'] = max_spent_city
df4['Min_spent_city'] = min_spent_city



,Country_Code,Country,Currency_code
0,1,India,INR
1,14,Australia,AUD
2,30,Brazil,BRL
3,37,Canada,CAD
4,94,Indonesia,IDR
5,148,New Zealand,NZD
6,162,Phillipines,PHP
7,166,Qatar,QAR
8,184,Singapore,SGD
9,189,South Africa,ZAR


,Country,Average_Money_Spent,Dine-In,Online_Delivery,Popular_Cusine,Costlier_Cusine,Max_spent_city,Min_spent_city
0,Australia,1369.761331,0,0,"Pizza, Bar Food",Modern Australian,Paynesville,Dicky Beach
1,Brazil,2149.504893,0,0,"Brazilian, Bar Food",Italian,S恚o Paulo,Bras韄lia
2,Canada,2205.092285,0,0,"Italian, Mediterranean, Pizza","Italian, Mediterranean, Pizza",Vineland Station,Chatham-Kent
3,India,623.370319,1111,2423,"North Indian, Chinese",European,New Delhi,Mohali
4,Indonesia,1520.396905,0,0,"Sunda, Indonesian","Italian, Continental",Jakarta,Bandung
5,New Zealand,3634.661201,0,0,Desserts,"French, Kiwi",Wellington City,Auckland
6,Phillipines,2395.790011,14,0,"Filipino, Mexican","European, Asian, Indian",Pasay City,Quezon City
7,Qatar,4999.787277,1,0,Chinese,International,Doha,Doha
8,Singapore,9617.929759,0,0,Bakery,"French, Mediterranean, European",Singapore,Singapore
9,South Africa,2035.949692,2,0,"European, Contemporary",French,Pretoria,Randburg


In [ ]:
# https://raw.githubusercontent.com/nethajinirmal13/Training-datasets/main/zomato/zomato.csv

import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/JaniceLibbyThomas/Sample_Dataset/main/Zomato.csv')
# df.to_csv('zomato.csv')
# display(df.head(10))

df1 = pd.read_csv('/content/Country-Code.csv')
display(df1)



In [ ]:
#checking for the missing value

df.isna().sum()

In [4]:
!pip install easy_exchange_rates 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
#finding exchange rate

from datetime import date

from easy_exchange_rates import API
api = API()
currency_conversion = list(df1['Currency_code'])
# print(currency_conversion)
indian_exchange_rate =[]

for i in range (0,len(currency_conversion)):
  time_series = api.get_exchange_rates(
  base_currency=currency_conversion[i], 
  start_date=str(date.today()), 
  end_date=str(date.today()), 
  targets=["INR"]
)
  data_frame = api.to_dataframe(time_series)
  indian_exchange_rate.append((time_series[str(date.today())]['INR']))
# print(data_frame.head(5))
# print(indian_exchange_rate)
df1['indian_exchange_rate']=indian_exchange_rate
df1

,Country_Code,Country,Currency_code,indian_exchange_rate
0,1,India,INR,1.000000
1,14,Australia,AUD,56.875903
2,30,Brazil,BRL,15.961670
3,37,Canada,CAD,60.830132
4,94,Indonesia,IDR,0.005407
5,148,New Zealand,NZD,52.109838
6,162,Phillipines,PHP,1.491015
7,166,Qatar,QAR,22.345418
8,184,Singapore,SGD,61.752358
9,189,South Africa,ZAR,4.850579


In [6]:
# creating a new column for country based on country code

# saved the dataframe as df2 after merging with the country code.

df3 = pd.merge(df,df1)
df3

,Restaurant_ID,Restaurant_Name,Country_Code,City,Address,Locality,Locality_Verbose,Longitude,Latitude,Cuisines,...,Is_delivering_now,Switch_to_order_menu,Price_range,Aggregate_rating,Rating_color,Rating_text,Votes,Country,Currency_code,indian_exchange_rate
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,No,No,3,4.8,Dark Green,Excellent,314,Phillipines,PHP,1.491015
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,No,No,3,4.5,Dark Green,Excellent,591,Phillipines,PHP,1.491015
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,No,No,4,4.4,Green,Very Good,270,Phillipines,PHP,1.491015
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",...,No,No,4,4.9,Dark Green,Excellent,365,Phillipines,PHP,1.491015
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",...,No,No,4,4.8,Dark Green,Excellent,229,Phillipines,PHP,1.491015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9546,5915730,Naml郾 Gurme,208,哿stanbul,"Kemanke侓 Karamustafa Pa侓a Mahallesi, R郾ht郾m Ca...",Karak韄y,"Karak韄y, 哿stanbul",28.977392,41.022793,Turkish,...,No,No,3,4.1,Green,Very Good,788,Turkey,TRY,4.330072
9547,5908749,Ceviz A埕ac郾,208,哿stanbul,"Ko侓uyolu Mahallesi, Muhittin 盱st韄nda埕 Caddesi,...",Ko侓uyolu,"Ko侓uyolu, 哿stanbul",29.041297,41.009847,"World Cuisine, Patisserie, Cafe",...,No,No,3,4.2,Green,Very Good,1034,Turkey,TRY,4.330072
9548,5915807,Huqqa,208,哿stanbul,"Kuru韄e侓me Mahallesi, Muallim Naci Caddesi, No ...",Kuru韄e侓me,"Kuru韄e侓me, 哿stanbul",29.034640,41.055817,"Italian, World Cuisine",...,No,No,4,3.7,Yellow,Good,661,Turkey,TRY,4.330072
9549,5916112,A侓侓k Kahve,208,哿stanbul,"Kuru韄e侓me Mahallesi, Muallim Naci Caddesi, No ...",Kuru韄e侓me,"Kuru韄e侓me, 哿stanbul",29.036019,41.057979,Restaurant Cafe,...,No,No,4,4.0,Green,Very Good,901,Turkey,TRY,4.330072


In [17]:
ex_rate = list(df3['indian_exchange_rate'])
# print(ex_rate)

rate_conversion = list(df3['Average_Cost_for_two'])
# print(rate_conversion)

Price_in_INR = []
for i in range (0,len(rate_conversion)):
  Price_in_INR.append(ex_rate[i]*rate_conversion[i])

df3['Indian_Rupee']=Price_in_INR
# df3

**DASH BOARD INSTALLATION**

In [9]:
!pip install jupyter_dash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 53.2 MB/s eta 0:00:00


**Sample Testing**

In [19]:
specific_country=(df3[ df3['Country']=='Australia'])
max_spent_city = []
min_spent_city = []
min_spent_amount = []
max_spent_amount = []
# --> Citywise data --> sum of cost --> store dic as city and sum amt
city = specific_country.City.unique()
city_wise_cost = (dict(specific_country.groupby("City")['Indian_Rupee'].sum()))
max_spent_city.append(max(zip(city_wise_cost.values(), city_wise_cost.keys()))[1])
print(max_spent_city)
print(city_wise_cost)
print(city_wise_cost[max_spent_city[0]])
max_spent_amount.append(city_wise_cost[(max_spent_city[0])])
#Min Spending City

min_spent_city.append(min(zip(city_wise_cost.values(), city_wise_cost.keys()))[1])
min_spent_amount.append(city_wise_cost[(min_spent_city[0])])

# print(max_spent_city,max_spent_amount,min_spent_city,min_spent_amount)

['Paynesville']
{'Armidale': 1133.9240599999998, 'Balingup': 1133.9240599999998, 'Beechworth': 1133.9240599999998, 'Dicky Beach': 396.873421, 'East Ballina': 1133.9240599999998, 'Flaxton': 1700.88609, 'Forrest': 1133.9240599999998, 'Hepburn Springs': 1530.7974809999998, 'Huskisson': 1133.9240599999998, 'Inverloch': 396.873421, 'Lakes Entrance': 396.873421, 'Lorn': 1133.9240599999998, 'Macedon': 1133.9240599999998, 'Mayfield': 1133.9240599999998, 'Middleton Beach': 1700.88609, 'Montville': 1700.88609, 'Palm Cove': 1700.88609, 'Paynesville': 6803.54436, 'Penola': 1133.9240599999998, 'Phillip Island': 1133.9240599999998, 'Tanunda': 1700.88609, 'Trentham East': 1133.9240599999998, 'Victor Harbor': 1133.9240599999998}
6803.54436


**Creating a new DF which has all the details on Countrywise**

In [ ]:
#calculation of average money spent by each country,dine in, delivery,costlier cusine, most popular cusine...

s = list(dict(df3.groupby("Country")['Indian_Rupee'].mean()))
s1 = list(df3.groupby("Country")['Indian_Rupee'].mean())

df4 = pd.DataFrame(s,columns=['Country'])
df4['Average_Money_Spent']=s1

Country_list = list(df4['Country'])
length = len(Country_list)


dine = []
delivery = []
Costlier_Cusine = []
Popular_Cusine = []
max_spent_city = []
min_spent_city = []
min_spent_amount = []
a1 = ''

for i in range(0,length):

#countrywise dataFrame
  specific_country=(df3[ df3['Country']==Country_list[i]])

# to get the dine-in and delivery count of each country
  dine.append(len(list(specific_country[specific_country['Has_Table_booking']=='Yes'].index.values)))
  delivery.append(len(list(specific_country[specific_country['Has_Online_delivery']=='Yes'].index.values)))

#most Costlier cusine

  max_spent = specific_country.Indian_Rupee.max()

  index = list(specific_country[specific_country['Indian_Rupee']==max_spent].index.values)

  Costlier_Cusine.append(specific_country.loc[int(index[0]),'Cuisines'])

#Most popular Cusine

  popular_vote = specific_country.Aggregate_rating.max()

  li = {}

  index = (specific_country[specific_country['Aggregate_rating']==popular_vote].index.values)

  for i in range(0,len(index)):
  # li[int(specific_country.loc[int(index[i]),['Votes']])]=(dict(specific_country.loc[int(index[i]),['Cuisines']]))
    b = (list(specific_country.loc[int(index[i]),['Votes']]))
    a = str(list(specific_country.loc[int(index[i]),['Cuisines']])).replace('[','').replace(']','').replace("'",'')
    li[a] = b[0]

  value = dict(li)

  Popular_Cusine.append(max(zip(value.values(), value.keys()))[1])

#Highest Spending City 

  city = specific_country.City.unique()
  city_wise_cost = (dict(specific_country.groupby("City")['Indian_Rupee'].sum()))
  max_spent_city.append(max(zip(city_wise_cost.values(), city_wise_cost.keys()))[1])

#Min Spending City

  min_spent_city.append(min(zip(city_wise_cost.values(), city_wise_cost.keys()))[1])
  # min_spent_amount.append(city_wise_cost[(min_spent_city[0])])

# Appending the data to Df4
# li1 = a1.split(' ')
df4['Dine-In'] = dine
df4['Online_Delivery'] = delivery
df4['Popular_Cusine'] = Popular_Cusine
df4['Costlier_Cusine'] = Costlier_Cusine
df4['Max_spent_city'] = max_spent_city
df4['Min_spent_city'] = min_spent_city
# df4['min_spent_amount'] = min_spent_amount
# df4['max_spent_amount'] = li1

df4
# df4

In [22]:
columns = []
# columns = df4['Popular_Cusine']#,'Costlier_Cusine','max_spent_city','min_spent_city']
columns_index = (df4[df4['Country']=='India'].index.values)
temp = ['Popular_Cusine','Costlier_Cusine','Max_spent_city','Min_spent_city']
for i in range(0,len(temp)):
  columns.append(df4.iloc[columns_index[0]][temp[i]])
columns
# columns


['North Indian, Chinese', 'European', 'New Delhi', 'Mohali']

#working code - country wise analysis

In [18]:
from dash import Dash, dcc, html, Input, Output
from jupyter_dash import JupyterDash
import dash_table as dt
import plotly.express as px
from plotly.subplots import make_subplots

app = JupyterDash(__name__)
sample_data = df4.to_dict('records')

length_record = len(df4.to_dict('records'))
temp_d = {}

for i in range (0,length_record):
  temp_d[sample_data[i]['Country']] = sample_data[i]

app.layout = html.Div(children = [
    html.H2('COUNTRY BASED ANALYSIS REPORT',style={
           'textAlign': 'center',
           'color': 'black',
           'font-family':'Comic Sans MS',
           'background-color':'#FFDFD3'
       }),
    dcc.Dropdown(
        list(df3['Country'].unique()),
        'India',
        id="dropdown"
    ),
    
    dt.DataTable(id='data-table', columns=[
           {'name': 'Popular_Cusine', 'id': 'Popular_Cusine'},
           {'name': 'Costlier_Cusine', 'id': 'Costlier_Cusine'},
           {'name': 'Max_spent_city', 'id': 'Max_spent_city'},
           {'name': 'Min_spent_city', 'id': 'Min_spent_city'},
       ],
    style_header={
        'backgroundColor': 'rgb(30, 30, 30)',
        'color': 'white'
    },
    style_data={
        'backgroundColor': 'rgb(50, 50, 50)',
        'color': 'white'
    }),
    dcc.Graph(id="graph"),
    
], id='container')


@app.callback(
    Output('data-table', 'data'),
    Output("graph", "figure"),
    
    Input("dropdown", "value"))

def update_bar_chart(value):

#getting the popular & costlier cusine,min and max spent city

    columns = []
    columns_index = (df4[df4['Country']==value].index.values)
    temp = ['Popular_Cusine','Costlier_Cusine','Max_spent_city','Min_spent_city']
    for i in range(0,len(temp)):
      columns.append(df4.iloc[columns_index[0]][temp[i]])
    dummy = pd.DataFrame([columns], columns = temp)
    data=dummy.to_dict('records')


#graph 

    fig = make_subplots(rows=1, cols=2,subplot_titles=('Average Money Spent',  'Dine-In VS Online Delivery'))

    avg_spent = int(df4.iloc[columns_index[0]]['Average_Money_Spent'])
    column_1 = []
    temp1 = ['Dine-In','Online_Delivery']
    for i in range(0,len(temp1)):
      column_1.append(df4.iloc[columns_index[0]][temp1[i]])

    fig.add_scatter(x =[f'{value}'],y=[avg_spent],
            marker=dict(color="#FFDFD3"),
            name="Indian Rupee ₹", row=1, col=1)

    fig.add_bar(x =['Dine-in','Online Delivery'],y= column_1,
            marker=dict(color="#FFDFD3"),
            name="Count", row=1, col=2)

    fig.update_layout(height=500, 
                      width= 1200,
                      showlegend=False,  plot_bgcolor='#FFDFD3',
                      paper_bgcolor="white",
                      font_color='#323232',
                      font_family="Comic Sans MS")

    fig.update_traces(overwrite=True, marker={"opacity": 0.5})
    
    fig.data[0].marker.line.width = 4
    fig.data[0].marker.line.color = "black"

    fig.data[1].marker.line.width = 4
    fig.data[1].marker.line.color = "black"


    return data,fig

if __name__ == '__main__':
   app.run_server(debug=True)

Dash app running on:


<IPython.core.display.Javascript object>

#sample working code

In [59]:
from dash import Dash, dcc, html, Input, Output
from jupyter_dash import JupyterDash
import dash_table as dt
import plotly.express as px

app = JupyterDash(__name__, suppress_callback_exceptions=True,external_stylesheets='https://raw.githubusercontent.com/JaniceLibbyThomas/Sample_Dataset/main/typography.css')
app.scripts.config.serve_locally = False
sample_data = df4.to_dict('records')

length_record = len(df4.to_dict('records'))
temp_d = {}

for i in range (0,length_record):
  temp_d[sample_data[i]['Country']] = sample_data[i]

tabs_styles = {
    'height': '44px'
}
tab_style = {
    'borderBottom': '1px solid #d6d6d6',
    'padding': '6px',
    'fontWeight': 'bold',
    'textAlign': 'center',
    'color': 'black',
    'font-family':'Comic Sans MS',
           
}

tab_selected_style = {
    'borderTop': '1px solid #d6d6d6',
    'borderBottom': '1px solid #d6d6d6',
    'backgroundColor': '#98BDFF',
    'color': 'white',
    'padding': '6px',
    'textAlign': 'center',
    'fontWeight': 'bold',
    # 'color': 'black',
    'font-family':'Comic Sans MS',
}


app.layout = html.Div([
    html.H2('ZOMATO ANALYSIS DASHBOARD',style={
           'textAlign': 'center',
           'color': 'white',
           'font-family':'Comic Sans MS',
           'background-color':'black'
       }),
    dcc.Tabs(id="tabs-styled-with-inline", value='tab-1', children=[
        dcc.Tab(label='Country-Wise Report', value='tab-1', style=tab_style, selected_style=tab_selected_style),
        dcc.Tab(label='City Wise Report', value='tab-2', style=tab_style, selected_style=tab_selected_style),
          ], style=tabs_styles),
    html.Div(id='tabs-content-inline'),
   
      
    
], id='container', style={'display': 'flex', 'flex-direction': 'column','background-color':'#4B49AC'})

@app.callback(Output('tabs-content-inline', 'children'),
              Input('tabs-styled-with-inline', 'value'),suppress_callback_exceptions=True)
def render_content(tab):
    if tab == 'tab-1':
        return  (html.Div(children=[
                dcc.Dropdown(
                list(df3['Country'].unique()),
                'India',
                id="dropdown",
                style={
                        'width': '100%'
                }
                ),
                html.Br(),
                dt.DataTable(id='data-table', columns=[
           {'name': 'Popular_Cusine', 'id': 'Popular_Cusine'},
           {'name': 'Costlier_Cusine', 'id': 'Costlier_Cusine'},
           {'name': 'Max_spent_city', 'id': 'Max_spent_city'},
           {'name': 'Min_spent_city', 'id': 'Min_spent_city'},
                ],
                style_header={
                'backgroundColor': 'green',
                'color': 'black'
                 },
                style_data={
                'backgroundColor': 'green',
                'color': 'black',
                'whiteSpace': 'normal',
                'height': 'auto',
                },
                fill_width=False),
                dcc.Graph(id="graph")]))
    elif tab == 'tab-2':
        return html.Div([
            html.H3('Tab content 2')
        ])
    

@app.callback(
    Output('data-table', 'data'),
    Output("graph", "figure"),   
    Input("dropdown", "value"),suppress_callback_exceptions=True)

def update_bar_chart(value):

    
#getting the popular & costlier cusine,min and max spent city
      columns = []
      columns_index = (df4[df4['Country']==value].index.values)
      temp = ['Popular_Cusine','Costlier_Cusine','Max_spent_city','Min_spent_city']
      for i in range(0,len(temp)):
        columns.append(df4.iloc[columns_index[0]][temp[i]])
      dummy = pd.DataFrame([columns], columns = temp)
      data=dummy.to_dict('records')


#graph 

      fig = make_subplots(rows=1, cols=2,subplot_titles=('Average Money Spent',  'Dine-In VS Online Delivery'))

      avg_spent = int(df4.iloc[columns_index[0]]['Average_Money_Spent'])
      column_1 = []
      temp1 = ['Dine-In','Online_Delivery']
      for i in range(0,len(temp1)):
        column_1.append(df4.iloc[columns_index[0]][temp1[i]])

      fig.add_scatter(x =[f'{value}'],y=[avg_spent],
            marker=dict(color="#FFDFD3"),
            name="Indian Rupee ₹", row=1, col=1)

      fig.add_bar(x =['Dine-in','Online Delivery'],y= column_1,
            marker=dict(color="#FFDFD3"),
            name="Count", row=1, col=2)

      fig.update_layout(height=500, 
                      width= 1200,
                      showlegend=False,  plot_bgcolor='#FFDFD3',
                      paper_bgcolor="white",
                      font_color='#323232',
                      font_family="Comic Sans MS")

      fig.update_traces(overwrite=True, marker={"opacity": 0.5})
    
      fig.data[0].marker.line.width = 4
      fig.data[0].marker.line.color = "black"

      fig.data[1].marker.line.width = 4
      fig.data[1].marker.line.color = "black"


      return data,fig

if __name__ == '__main__':
   app.run_server(debug=True)

Dash app running on:


<IPython.core.display.Javascript object>

In [ ]:
from dash import Dash, dcc, html, Input, Output
from jupyter_dash import JupyterDash
import dash_table as dt
import plotly.express as px
from plotly.subplots import make_subplots

columns = []
value = 'India'
columns_index = (df4[df4['Country']==value].index.values)
temp = ['Popular_Cusine','Costlier_Cusine','Max_spent_city','Min_spent_city']
for i in range(0,len(temp)):
  columns.append(df4.iloc[columns_index[0]][temp[i]])
dummy = pd.DataFrame([columns], columns = temp)
data=dummy.to_dict('records')

print(data)
#graph 

fig = make_subplots(rows=1, cols=2,subplot_titles=('Average Money Spent',  'Dine-In VS Online Delivery'))

avg_spent = int(df4.iloc[columns_index[0]]['Average_Money_Spent'])
column_1 = []
temp1 = ['Dine-In','Online_Delivery']
for i in range(0,len(temp1)):
  column_1.append(df4.iloc[columns_index[0]][temp1[i]])

fig.add_scatter(x =[f'{value}'],y=[avg_spent],
marker=dict(color="#FFDFD3"),
name="Indian Rupee ₹", row=1, col=1)

fig.add_bar(x =['Dine-in','Online Delivery'],y= column_1,
marker=dict(color="#FFDFD3"),
name="Count", row=1, col=2)

fig.update_layout(height=500, 
  width= 1200,
  showlegend=False,  plot_bgcolor='#FFDFD3',
  paper_bgcolor="white",
  font_color='#323232',
  font_family="Comic Sans MS")

fig.update_traces(overwrite=True, marker={"opacity": 0.5})

fig.data[0].marker.line.width = 4
fig.data[0].marker.line.color = "black"

fig.data[1].marker.line.width = 4
fig.data[1].marker.line.color = "black"

fig

In [ ]:
columns = []
value ='India'
columns_index = (df4[df4['Country']==value].index.values)
temp = ['Popular_Cusine','Costlier_Cusine','Max_spent_city','Min_spent_city']
# print(columns_index)
for i in range(0,len(temp)):
  columns.append(df4.iloc[columns_index[0]][temp[i]])
# dummy = pd.DataFrame([columns], columns = temp,index = ['v']).T
d1 = pd.DataFrame(temp,columns = ['Categories'])
d1['Values'] = columns
d1


# data=dummy.to_dict('records')

In [ ]:
!pip install dash-bootstrap-templates

**Template Sample 1 - with Side Bar**

In [64]:
from dash import Dash, dcc, html, dash_table, Input, Output, callback
import plotly.express as px
import dash_bootstrap_components as dbc
from dash_bootstrap_templates import ThemeChangerAIO, template_from_url

df = px.data.gapminder()
# years = df.year.unique()
city = df4.Country.unique()

a = list(df3['City'])
b = list(df3['Country'])
new_list = []

for i in range (0,len(a)):
  new_list.append(a[i]+','+b[i])

df3['City_Countrywise']=new_list

columns_2 = []
# value = 'India'
columns_index1 = (df3[df3['Country']==value].index.values)

for i in range(0,len(columns_index)):
  columns_2.append(df3.iloc[columns_index1[i]]['City_Countrywise'])
data_value = list(set(columns))

# stylesheet with the .dbc class
dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP, dbc_css])

header = html.H4(
    "Janice ~ Zomato Analysis Dashboard", className="bg-primary text-white p-2 mb-2 text-center"
)

table = html.Div(
    dash_table.DataTable(
        id="table",
        columns=[{"name": i, "id": i, "deletable": True} for i in df4.columns],
        data=df4.to_dict("records"),
        page_size=10,
        editable=True,
        cell_selectable=True,
        filter_action="native",
        sort_action="native",
        style_table={"overflowX": "auto"},
        row_selectable="multi",
    ),
    className="dbc-row-selectable",
)

dropdown = html.Div(
    [
        dbc.Label("Select a Country"),
        dcc.Dropdown(
            df4.Country.unique(),
            "India",
            id="indicator",
            clearable=False,
        ),
    ],
    className="mb-4",
)

dropdown1 = html.Div(
    [
        dbc.Label("Select a City"),
        dcc.Dropdown(
            data_value,
            "Agra,India",
            id="indicator1",
            clearable=False,
        ),
    ],
    className="mb-4",
)

# checklist = html.Div(
#     [
#         dbc.Label("Select City"),
#         dbc.Checklist(
#             id="City",
#             options=[{"label": i, "value": i} for i in city],
#             value=city,
#             inline=True,
#         ),
#     ],
#     className="mb-4",
# )

# slider = html.Div(
#     [
#         dbc.Label("Select Years"),
#         dcc.RangeSlider(
#             years[0],
#             years[-1],
#             5,
#             id="years",
#             marks=None,
#             tooltip={"placement": "bottom", "always_visible": True},
#             value=[years[2], years[-2]],
#             className="p-0",
#         ),
#     ],
#     className="mb-4",
# )
# theme_colors = [
#     "primary",
#     "secondary",
#     "success",
#     "warning",
#     "danger",
#     "info",
#     "light",
#     "dark",
#     "link",
# ]
# colors = html.Div(
#     [dbc.Button(f"{color}", color=f"{color}", size="sm") for color in theme_colors]
# )
# colors = html.Div(["Theme Colors:", colors], className="mt-2")


controls = dbc.Card(
    [dropdown, dropdown1],
    body=True,
)

tab1 = dbc.Tab([dcc.Graph(id="line-chart")], label="Line Chart")
tab2 = dbc.Tab([dcc.Graph(id="scatter-chart")], label="Scatter Chart")
tab3 = dbc.Tab([table], label="Table", className="p-4")
tabs = dbc.Card(dbc.Tabs([tab1, tab2, tab3]))

app.layout = dbc.Container(
    [
        header,
        dbc.Row(
            [
                dbc.Col(
                    [
                        controls,
                        # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
                        # When running this app locally, un-comment this line:
                        # ThemeChangerAIO(aio_id="theme")
                    ],
                    width=4,
                ),
                dbc.Col([tabs, colors], width=8),
            ]
        ),
    ],
    fluid=True,
    className="dbc",
)


@callback(
    Output("line-chart", "figure"),
    Output("scatter-chart", "figure"),
    Output("table", "data"),
    Input("indicator", "value"),
    Input("indicator1", "value"),
    Input("continents", "value"),
    Input("years", "value"),
    Input(ThemeChangerAIO.ids.radio("theme"), "value"),
)
def update_line_chart(indicator,indicator1, continent, yrs, theme):
    if continent == [] or indicator is None:
        return {}, {}, []

    dff = df[df.year.between(yrs[0], yrs[1])]
    dff = dff[dff.continent.isin(continent)]
    data = dff.to_dict("records")

    fig = px.line(
        dff,
        x="year",
        y=indicator,
        color="continent",
        line_group="country",
        template=template_from_url(theme),
    )

    fig_scatter = px.scatter(
        df.query(f"year=={yrs[1]} & continent=={continent}"),
        x="gdpPercap",
        y="lifeExp",
        size="pop",
        color="continent",
        log_x=True,
        size_max=60,
        template=template_from_url(theme),
        title="Gapminder %s: %s theme" % (yrs[1], template_from_url(theme)),
    )

    return fig, fig_scatter, data


if __name__ == "__main__":
  app.run_server(debug=True)

Dash app running on:


<IPython.core.display.Javascript object>

In [ ]:
a = list(df3['City'])
b = list(df3['Country'])
new_list = []

for i in range (0,len(a)):
  new_list.append(a[i]+','+b[i])

df3['City_Countrywise']=new_list
df3

columns_2 = []
value = 'India'
columns_index1 = (df3[df3['Country']==value].index.values)

for i in range(0,len(columns_index)):
  columns_2.append(df3.iloc[columns_index1[i]]['City_Countrywise'])

data_value = list(set(columns))

**Template No 2: Updating dropdown value based on selection**

In [11]:
import dash
import dash_core_components as dcc
import dash_html_components as html

app = JupyterDash()

fnameDict = {'Country': list(df4['Country']), 'City': ['opt1_j', 'opt2_j']}

names = list(fnameDict.keys())
nestedOptions = fnameDict[names[0]]
print(names)
# app.layout = html.Div(
#     [
#         html.Div([
#         dcc.Dropdown(
#             id='name-dropdown',
#             options=[{'label':name, 'value':name} for name in names],
#             value = list(fnameDict.keys())[0]
#             ),
#             ],style={'width': '20%', 'display': 'inline-block'}),
#         html.Div([
#         dcc.Dropdown(
#             id='opt-dropdown',
#             ),
#             ],style={'width': '20%', 'display': 'inline-block'}
#         ),
#         html.Hr(),
#         html.Div(id='display-selected-values')
#     ]
# )

# @app.callback(
#     dash.dependencies.Output('opt-dropdown', 'options'),
#     [dash.dependencies.Input('name-dropdown', 'value')]
# )
# def update_date_dropdown(name):
#     return [{'label': i, 'value': i} for i in fnameDict[name]]

# @app.callback(
#     dash.dependencies.Output('display-selected-values', 'children'),
#     [dash.dependencies.Input('opt-dropdown', 'value')])
# def set_display_children(selected_value):
#     return 'you have selected {} option'.format(selected_value)



# if __name__ == '__main__':
#     app.run_server()

['chriddy', 'jackp']


<ipython-input-11-d4ed69053086>:2: UserWarning:


The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`

<ipython-input-11-d4ed69053086>:3: UserWarning:


The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`



In [ ]:
import dash_boostrap_components as dbc

app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
dbc.Col(children=[child1, child2,], lg=6, md=12)

In [150]:
# df3['City_with_Country'] = 
a = list(df3['City'])
print(a)
print(b)
b = list(df3['Country'])
new_list = []

for i in range (0,len(a)):
  new_list.append(a[i]+','+b[i])

du = pd.DataFrame(new_list)
du

['Makati City', 'Makati City', 'Mandaluyong City', 'Mandaluyong City', 'Mandaluyong City', 'Mandaluyong City', 'Pasay City', 'Pasay City', 'Pasay City', 'Pasig City', 'Pasig City', 'Pasig City', 'Quezon City', 'San Juan City', 'San Juan City', 'Santa Rosa', 'Santa Rosa', 'Tagaytay City', 'Taguig City', 'Taguig City', 'Taguig City', 'Taguig City', 'Bras韄lia', 'Bras韄lia', 'Bras韄lia', 'Bras韄lia', 'Bras韄lia', 'Bras韄lia', 'Bras韄lia', 'Bras韄lia', 'Bras韄lia', 'Bras韄lia', 'Bras韄lia', 'Bras韄lia', 'Bras韄lia', 'Bras韄lia', 'Bras韄lia', 'Bras韄lia', 'Bras韄lia', 'Bras韄lia', 'Bras韄lia', 'Bras韄lia', 'Rio de Janeiro', 'Rio de Janeiro', 'Rio de Janeiro', 'Rio de Janeiro', 'Rio de Janeiro', 'Rio de Janeiro', 'Rio de Janeiro', 'Rio de Janeiro', 'Rio de Janeiro', 'Rio de Janeiro', 'Rio de Janeiro', 'Rio de Janeiro', 'Rio de Janeiro', 'Rio de Janeiro', 'Rio de Janeiro', 'Rio de Janeiro', 'Rio de Janeiro', 'Rio de Janeiro', 'Rio de Janeiro', 'Rio de Janeiro', 'S恚o Paulo', 'S恚o Paulo', 'S恚o Paulo', 'S恚o Paulo',

,0
0,"Makati City,Phillipines"
1,"Makati City,Phillipines"
2,"Mandaluyong City,Phillipines"
3,"Mandaluyong City,Phillipines"
4,"Mandaluyong City,Phillipines"
...,...
9546,"哿stanbul,Turkey"
9547,"哿stanbul,Turkey"
9548,"哿stanbul,Turkey"
9549,"哿stanbul,Turkey"


In [154]:
from dash import Dash, dcc, html, Input, Output
from jupyter_dash import JupyterDash
import dash_table as dt
import plotly.express as px

app = JupyterDash(__name__)

a = list(df3['City'])
b = list(df3['Country'])
country_City = []

for i in range (0,len(a)):
  country_City.append(a[i]+','+b[i])

df3['Country_City'] = country_City

app.layout = html.Div([
    html.H2('COUNTRY BASED ANALYSIS REPORT',style={
           'textAlign': 'center',
           'color': 'black',
           'font-family':'Comic Sans MS',
           'background-color':'#FFDFD3'
       }),
    dcc.Dropdown(
        list(df3['Country'].unique()),
        'India',
        id="dropdown"
    ),
    
    dt.DataTable(id='data-table', columns=[
           {'name': 'Popular_Cusine', 'id': 'Popular_Cusine'},
           {'name': 'Costlier_Cusine', 'id': 'Costlier_Cusine'},
           {'name': 'Max_spent_city', 'id': 'Max_spent_city'},
           {'name': 'Min_spent_city', 'id': 'Min_spent_city'},
       ],
    style_header={
        'backgroundColor': 'rgb(30, 30, 30)',
        'color': 'white'
    },
    style_data={
        'backgroundColor': 'rgb(50, 50, 50)',
        'color': 'white'
    }),
    dcc.Graph(id="graph"),

    dcc.Dropdown(
        list(df3['Country_City'].unique()),
        'Agra,India',
        id="dropdown1"
    ),
    # dcc.Graph(id="graph1"),  
    
], id='container')


@app.callback(
    Output('data-table', 'data'),
    Output("graph", "figure"),
    # Output("graph1", "figure"),
    Input("dropdown", "value"),
    Input("dropdown1", "value1"))

def update_bar_chart(value):

#getting the popular & costlier cusine,min and max spent city

    columns = []
    columns_index = (df4[df4['Country']==value].index.values)
    temp = ['Popular_Cusine','Costlier_Cusine','Max_spent_city','Min_spent_city']
    for i in range(0,len(temp)):
      columns.append(df4.iloc[columns_index[0]][temp[i]])
    dummy = pd.DataFrame([columns], columns = temp)
    data=dummy.to_dict('records')


#graph 

    fig = make_subplots(rows=1, cols=2,subplot_titles=('Average Money Spent',  'Dine-In VS Online Delivery'))

    avg_spent = int(df4.iloc[columns_index[0]]['Average_Money_Spent'])
    column_1 = []
    temp1 = ['Dine-In','Online_Delivery']
    for i in range(0,len(temp1)):
      column_1.append(df4.iloc[columns_index[0]][temp1[i]])

    fig.add_scatter(x =[f'{value}'],y=[avg_spent],
            marker=dict(color="#FFDFD3"),
            name="Indian Rupee ₹", row=1, col=1)

    fig.add_bar(x =['Dine-in','Online Delivery'],y= column_1,
            marker=dict(color="#FFDFD3"),
            name="Count", row=1, col=2)

    fig.update_layout(height=500, 
                      width= 1200,
                      showlegend=False,  plot_bgcolor='#FFDFD3',
                      paper_bgcolor="white",
                      font_color='#323232',
                      font_family="Comic Sans MS")

    fig.update_traces(overwrite=True, marker={"opacity": 0.5})
    
    fig.data[0].marker.line.width = 4
    fig.data[0].marker.line.color = "black"

    fig.data[1].marker.line.width = 4
    fig.data[1].marker.line.color = "black"


    return data,fig

if __name__ == '__main__':
   app.run_server(debug=True)

Dash app running on:


<IPython.core.display.Javascript object>

In [11]:
sample_data = {
   'series': {
       'data': [
           {'title': 'Game of Thrones', 'score': 9.5},
           {'title': 'Stranger Things', 'score': 8.9},
           {'title': 'Vikings', 'score': 8.6}
       ],
       'style': {
           'backgroundColor': 'white'
       }
   },
   'movies': {
       'data': [
           {'title': 'Rambo', 'score': 7.7},
           {'title': 'The Terminator', 'score': 8.0},
           {'title': 'Alien', 'score': 8.5}
       ],
       'style': {
           'backgroundColor': '#fff289'
       }
   }
}

# dt.DataTable(id='data-table', columns=[
#            {'name': 'Title', 'id': 'title'},
#            {'name': 'Score', 'id': 'score'}
#        ])

selected = sample_data['series']
data = selected['data']
columns = [
       {'name': k.capitalize(), 'id': k}
       for k in data[0].keys()
   ]
print(columns)

[{'name': 'Title', 'id': 'title'}, {'name': 'Score', 'id': 'score'}]


In [86]:
column_1 = []


columns_index = (df4[df4['Country']=='India'].index.values)
temp1 = ['Dine-In','Online_Delivery']
for i in range(0,len(temp1)):
  column_1.append(df4.iloc[columns_index[0]][temp1[i]])
# df4.iloc[columns_index[0]][temp1[1]]

column_1

[1111, 2423]

In [18]:
from dash import Dash, dcc, html, Input, Output
from jupyter_dash import JupyterDash
import plotly.tools as tls
import plotly.express as px

app = JupyterDash(__name__)

app.layout = html.Div([
    html.H4('ANALYSIS BASED ON COUNTRY'),
    dcc.Dropdown(
        id="dropdown",
        options=list(df3['Country'].unique()),
        placeholder="Select a Country"
        # value=['Country', 'sepal_width'],
        # multi=True


    ),
    dcc.Graph(id="graph"),
])


@app.callback(
    Output("graph", "figure"), 
    Input("dropdown", "value"))
def update_bar_chart(value):

    value1 = str(value)
    
    # value ='United States'
    index = str(df4[df4['Country']==value1].index.values).replace('[','').replace(']','')
    money_spent = (list(df4.iloc[int(index)])[1])
    fig = tls.make_subplots(rows=1, cols=2, shared_xaxes=True,vertical_spacing=0.0009,horizontal_spacing=0.0009,subplot_titles=("Average Money Spent", "Dine-in Vs Online Delivery"))
    # fig = px.bar(df4,x = [value1], y =[money_spent] ,title=f'Average Spent of <b>{value1}</b>',text_auto=True,labels=dict(x=f"<b>Country</b>", y="<b>Average_Spent</b>"),color=[list(df4.iloc[int(index)])[0]])
    fig.append_trace({'x':[value1,value1],'y':[money_spent,money_spent],'type':'bar','name':f'{value1}'},1,1)
    fig.append_trace({'x':[value1],'y':[money_spent],'type':'bar','name':f'{value1}'},1,2)
    return fig
    # return (value)

app.run_server(debug=True)

Dash app running on:


<IPython.core.display.Javascript object>

In [ ]:
import plotly.tools as tls
value ='brazil'
specific_country=(df3[ df3['Country']==value])

# (a[a['Has Table booking']=='Yes'].index.values)

index_dine = len(list(specific_country[specific_country['Has Table booking']=='Yes'].index.values))
# print(index_dine)

index_delivery = len(list(specific_country[specific_country['Has Online delivery']=='Yes'].index.values))
# print(index_delivery)

fig = tls.make_subplots(rows=1, cols=2, shared_xaxes=True,vertical_spacing=0.0009,horizontal_spacing=0.0009,subplot_titles=("Average Money Spent", "Dine-in Vs Online Delivery"))
    # fig = px.bar(df4,x = [value1], y =[money_spent] ,title=f'Average Spent of <b>{value1}</b>',text_auto=True,labels=dict(x=f"<b>Country</b>", y="<b>Average_Spent</b>"),color=[list(df4.iloc[int(index)])[0]])
fig.append_trace({'x':[index_dine],'y':[index_delivery],'type':'bar','name':f'{value}'},1,1)
fig.append_trace({'x':['Dine-in','Online Delivery'],'y':[index_dine,index_delivery],'type':'bar','name':f'{value}'},1,2)
(fig) 


In [ ]:
dine_in = df3.groupby("Country")['Has Table booking'].count()

Delivery = df3.groupby("Country")['Has Online delivery'].iloc()

print(dine_in)
print(Delivery)

Country
Australia           24
Brazil              60
Canada               4
India             8652
Indonesia           21
New Zealand         40
Phillipines         22
Qatar               20
Singapore           20
South Africa        60
Sri Lanka           20
Turkey              34
UAE                 60
United Kingdom      80
United States      434
Name: Has Table booking, dtype: int64
Country
Australia           24
Brazil              60
Canada               4
India             8652
Indonesia           21
New Zealand         40
Phillipines         22
Qatar               20
Singapore           20
South Africa        60
Sri Lanka           20
Turkey              34
UAE                 60
United Kingdom      80
United States      434
Name: Has Online delivery, dtype: int64


In [ ]:
fig = px.bar(df3,x=df3['Country'].unique(),y = df3.groupby("Country")['Indian_Rupee'].mean(),title='Average Spent of Each country',text_auto=True,labels=dict(x="<b>Country</b>", y="<b>Average_Spent</b>"))
fig

In [ ]:
#displaying values based on value given
value ='United States'
index = str(df4[df4['Country']==value].index.values).replace('[','').replace(']','')
money_spent = (list(df4.iloc[int(index)])[1])

fig = px.bar(df4,x = [value], y =[money_spent] ,title=f'Average Spent of <b>{value}</b>',text_auto=True,labels=dict(x=f"<b>Country</b>", y="<b>Average_Spent</b>"),color=[list(df4.iloc[int(index)])[0]])
fig


NameError: ignored

In [ ]:
fig = px.bar(df3,x=df3['Country'].unique(),y = df3.groupby("Country")['Indian_Rupee'].mean(),title='Average Spent of Each country',text_auto=True,labels=dict(x="<b>Country</b>", y="<b>Average_Spent</b>"))
fig

In [ ]:
#exporting the final dataset after data engineering concept

df3.to_csv('df3_Final.csv')

In [ ]:
# unique country name list

a = list(df3['Country'].unique())
a

['Phillipines',
 'Brazil',
 'United States',
 'Australia',
 'Canada',
 'Singapore',
 'UAE',
 'India',
 'Indonesia',
 'New Zealand',
 'United Kingdom',
 'Qatar',
 'South Africa',
 'Sri Lanka',
 'Turkey']

In [ ]:
# average spent of each country
b = []

for i in range (0,len(a)):

  b.append((df3.loc[df3['Country'] == a[i], 'Indian_Rupee'].mean()))

print(b)

[2368.324668181818, 2137.997088, 2121.8857951612904, 1361.9121869166665, 2198.4168474999997, 9556.050439249997, 3706.3874003333335, 623.370319001387, 1500.1511904761905, 3615.20855325, 4734.593415, 4983.300035, 2036.3165392, 524.217125, 366.2065416176471]


#changing the graph dynamically - PLOTLY EXPRESS

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# confirmed_global
confirmed_global='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'

covid19_confirmed=pd.read_csv(confirmed_global,index_col='Country/Region')

confirmed_latest = covid19_confirmed.T.index.values[-1]

df_grouped_conf=covid19_confirmed.groupby('Country/Region').sum()

df_confirmed=df_grouped_conf.sort_values(by=confirmed_latest,ascending=False).head(10).drop(['Lat', 'Long'],axis=1).T

def multi_plot(df, title, addAll = True):
    fig = go.Figure()

    for column in df.columns.to_list():
        fig.add_trace(
            go.Scatter(
                x = df.index,
                y = df[column],
                name = column
            )
        )

    button_all = dict(label = 'All',
                      method = 'update',
                      args = [{'visible': df.columns.isin(df.columns),
                               'title': 'All',
                               'showlegend':True}])

    def create_layout_button(column):
        return dict(label = column,
                    method = 'update',
                    args = [{'visible': df.columns.isin([column]),
                             'title': column,
                             'showlegend': True}])

    fig.update_layout(
        updatemenus=[go.layout.Updatemenu(
            active = 0,
            buttons = ([button_all] * addAll) + list(df.columns.map(lambda column: create_layout_button(column)))
            )
        ],
         yaxis_type="log"       
    )
    # Update remaining layout properties
    fig.update_layout(
        title_text=title,
        height=800
        
    )
   
    fig.show()
    
multi_plot(df_confirmed, title="Logarithmic COVID-19 time series total confirmed by country")